In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
from ggplot import *
plt.style.use('ggplot') 

import seaborn as sns

# for maps
from mapboxgl.utils import *
from mapboxgl.viz import *
import geojson

# Load the data

In [ ]:
file_path_historical = './data/Police_Department_Crash_Data_-_Historical.csv'
file_path_updated = './data/Police_Department_Crash_Data_-_Updated.csv'

data_historical = pd.read_csv(file_path_historical)
data_updated = pd.read_csv(file_path_updated)

# Data wrangling

In [ ]:
# Fix column names to match across datasets
data_historical = data_historical.rename(columns={"Day Of Week": "Day of Week", "Steet Name": "Street Name"})

In [ ]:
# Extract coordinates for each crash location
data_historical = data_historical.dropna(subset=['Latitude', 'Longitude'])
# If coordinates don't exist drop row
data_historical = data_historical.drop(columns=['Coordinates'])
# Drop duplicates, need to specify columns to match against because there are slight variations for some reason
data_historical = data_historical.drop_duplicates(subset=['Date Time', 'Day of Week', 'Object 1', 'Object 2'])

In [ ]:
# Drop rows without location
data_updated = data_updated.dropna(subset=['Location'])
# Drop rows without coordinates (these just use city center)
data_updated = data_updated.drop(data_updated[data_updated['Location'].apply(lambda x: len(x.split('\n')) != 3)].index)
# Create Coordinate column
data_updated['Coordinates'] = data_updated['Location'].apply(lambda x: x.split('\n')[2])
# Create Latitude and Longitude columns
data_updated['Latitude'] = data_updated['Coordinates'].apply(lambda x: float(x.split(',')[0].replace('(', '')))
data_updated['Longitude'] = data_updated['Coordinates'].apply(lambda x: float(x.split(',')[1].replace(')', '')))
# Drop the no longer needed Coordinates column
data_updated = data_updated.drop(columns=['Coordinates'])
# Drop duplicates, need to specify columns to match against because there are slight variations for some reason
data_updated = data_updated.drop_duplicates(subset=['Date Time', 'Day of Week', 'Object 1', 'Object 2'])

In [ ]:
# Combine datasets
data = pd.concat([data_historical, data_updated], ignore_index=True)

In [ ]:
# Filter for only the interesting and filled in data
data = data[['Date Time', 'Day of Week', 'Object 1', 'Object 2', 'Street Number', 'Street Name', 'Cross Street', 'Location', 'Latitude', 'Longitude', 'May Involve Pedestrian', 'May involve cyclist']]

In [ ]:
# Remove duplicates
# Caused by the two datasets overlapped reporting and lousy data
# Need to specify columns to match against because there are slight variations between the two datasets
# Keep the last rather than first since the updated dataset is more detailed, but worse coordinates
# data = data.drop_duplicates(subset=['Date Time', 'Day of Week', 'Object 1', 'Object 2'], keep='last')
data = data.groupby('Date Time').agg({
    'Day of Week': 'last',
    'Object 1': 'last',
    'Object 2': 'last',
    'Street Number': 'last',
    'Street Name': 'last',
    'Cross Street': 'last',
    'Location': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'May Involve Pedestrian': 'last',
    'May involve cyclist': 'last'
}).reset_index()

In [ ]:
# Convert data types for easier analysis
data['Date Time'] = data['Date Time'].apply(lambda x: pd.to_datetime(x))
# Create new columns for analysis
data['Coordinates'] = data.apply(lambda x: str(x["Latitude"]) + ',' + str(x["Longitude"]), axis=1)
data['Hour of Day'] = data['Date Time'].apply(lambda x: x.hour)
data['Year'] = data['Date Time'].apply(lambda x: x.year)
data['Month of Year'] = data['Date Time'].apply(lambda x: x.month)
data['Objects Involved'] = data.apply(lambda x: str(x["Object 1"]) + '-' + str(x["Object 2"]), axis=1)
data['Bicycle Involved'] = data.apply(lambda x: (x['Object 1'] == 'Bicycle') | (x['Object 2'] == 'Bicycle') | (x['May involve cyclist'] == True), axis=1)
data['Pedestrian Involved'] = data.apply(lambda x: (x['Object 1'] == 'Pedestrian') | (x['Object 2'] == 'Pedestrian') | (x['May Involve Pedestrian'] == True), axis=1)
data['No Bike or Pedestrian Involved'] = data.apply(lambda x: (x['Bicycle Involved'] == False) and (x['Pedestrian Involved'] == False), axis=1)
data['Date'] = data['Date Time'].apply(lambda x: pd.to_datetime(x.date()))

# Analysis

### Interesting questions
* Where did the accidents take place?
* Who were they between? Bicycles? Pedestrians?
* What time of day/day of week?
* Did they increase/decrease over time? Per location?
* Look at variables that could have made a difference (junction type, surface condition, street vs intersection, weather condition)

## Where do accidents take place?

In [ ]:
location_groups = data.groupby(['Latitude', 'Longitude'])
locations_df = location_groups.size().to_frame(name='# of accidents').reset_index()

In [ ]:
locations_df.sort_values(by=['# of accidents'], ascending=False).head()

## Who are they between?

In [ ]:
object_groups = data.groupby(['Objects Involved'])
objects_df = object_groups.size().to_frame(name='# of accidents').reset_index()

In [ ]:
objects_df.sort_values(by=['# of accidents'], ascending=False).head(15)

#### Bicycles

In [ ]:
bicycle_data = data[(data['Bicycle Involved'] == True)]
bicycle_groups = bicycle_data.groupby(['Objects Involved'])
bicycles_df = bicycle_groups.size().to_frame(name='# of accidents').reset_index()

In [ ]:
bicycles_df.sort_values(by=['# of accidents'], ascending=False).head(15)

#### Pedestrians

In [ ]:
pedestrian_data = data[(data['Pedestrian Involved'] == True)]
pedestrian_groups = pedestrian_data.groupby(['Objects Involved'])
pedestrians_df = pedestrian_groups.size().to_frame(name='# of accidents').reset_index()

In [ ]:
pedestrians_df.sort_values(by=['# of accidents'], ascending=False).head(15)

## When do accidents take place?

### Day of Week

In [ ]:
day_of_week_groups = data.groupby(['Day of Week'])
day_of_week_df = day_of_week_groups.size().to_frame(name='# of accidents').reset_index()
day_of_week_df.to_csv('output/day-of-week-all.csv')

In [ ]:
day_of_week_df.sort_values(by=['# of accidents'], ascending=False).head(7)

#### Bicycles

In [ ]:
day_of_week_groups_bicycle = bicycle_data.groupby(['Day of Week'])
day_of_week_bicycle_df = day_of_week_groups_bicycle.size().to_frame(name='# of accidents').reset_index()
day_of_week_bicycle_df.to_csv('output/day-of-week-bicycle.csv')

In [ ]:
day_of_week_bicycle_df.sort_values(by=['# of accidents'], ascending=False).head(7)

#### Pedestrians

In [ ]:
day_of_week_groups_pedestrian = pedestrian_data.groupby(['Day of Week'])
day_of_week_pedestrian_df = day_of_week_groups_pedestrian.size().to_frame(name='# of accidents').reset_index()
day_of_week_pedestrian_df.to_csv('output/day-of-week-pedestrian.csv')

In [ ]:
day_of_week_pedestrian_df.sort_values(by=['# of accidents'], ascending=False).head(7)

### Hour of Day

In [ ]:
time_ranges = pd.cut(data['Hour of Day'], [0, 4, 9, 13, 16, 20, 23], labels=['12am-5am', '5am-10am', '10am-1pm', '1pm-4pm', '4pm-8pm', '8pm-11:59pm'])

In [ ]:
data['Time Range'] = time_ranges

In [ ]:
time_range_groups = data.groupby(['Time Range'])
time_range_df = time_range_groups.size().to_frame(name="# of accidents").reset_index()
time_range_df.to_csv('output/time-ranges-all.csv')

#### Bicycles

In [ ]:
bicycle_time_ranges = pd.cut(bicycle_data['Hour of Day'], [0, 4, 9, 13, 16, 20, 23], labels=['12am-5am', '5am-10am', '10am-1pm', '1pm-4pm', '4pm-8pm', '8pm-11:59pm'])
bicycle_data['Time Range'] = bicycle_time_ranges

In [ ]:
bicycle_time_range_groups = bicycle_data.groupby(['Time Range'])
bicycle_time_range_df = bicycle_time_range_groups.size().to_frame(name="# of accidents").reset_index()
bicycle_time_range_df.to_csv('output/time-ranges-bicycles.csv')

#### Pedestrians

In [ ]:
pedestrian_time_ranges = pd.cut(pedestrian_data['Hour of Day'], [0, 4, 9, 13, 16, 20, 23], labels=['12am-5am', '5am-10am', '10am-1pm', '1pm-4pm', '4pm-8pm', '8pm-11:59pm'])
pedestrian_data['Time Range'] = pedestrian_time_ranges

In [ ]:
pedestrian_time_range_groups = pedestrian_data.groupby(['Time Range'])
pedestrian_time_range_df = pedestrian_time_range_groups.size().to_frame(name="# of accidents").reset_index()
pedestrian_time_range_df.to_csv('output/time-ranges-pedestrians.csv')

### How are they doing over time?

In [ ]:
# Annual Datasets
data_2010 = data[(data['Year'] == 2010)]
data_2011 = data[(data['Year'] == 2011)]
data_2012 = data[(data['Year'] == 2012)]
data_2013 = data[(data['Year'] == 2013)]
data_2014 = data[(data['Year'] == 2014)]
data_2015 = data[(data['Year'] == 2015)]
data_2016 = data[(data['Year'] == 2016)]
data_2017 = data[(data['Year'] == 2017)]

In [ ]:
accidents_by_day = data.groupby('Date').size()

In [ ]:
accidents_by_day_df = accidents_by_day.to_frame(name='# of accidents').reset_index()

In [ ]:
accidents_by_month = accidents_by_day.resample('M').sum()

In [ ]:
accidents_by_year = accidents_by_day.resample('Y').sum()

In [ ]:
accidents_by_day.sort_values(ascending=False).head()

In [ ]:
accidents_by_day.plot()

In [ ]:
accidents_by_month_df = accidents_by_month.to_frame(name='# of accidents').reset_index()
accidents_by_month_df.sort_values(['# of accidents'], ascending=False).head(10)

In [ ]:
accidents_by_month_df.to_csv('output/accidents-by-month-all.csv')

In [ ]:
accidents_by_month_of_year = data.groupby('Month of Year').size()
accidents_by_month_of_year_df = accidents_by_month_of_year.to_frame(name='# of accidents').reset_index()

In [ ]:
accidents_by_month_of_year.sort_values(ascending=False).head(12)

In [ ]:
accidents_by_month.plot()

In [ ]:
accidents_by_year.sort_values(ascending=False).head(10)

In [ ]:
accidents_by_year_groups = data.groupby('Year').size()
accidents_by_year_df = accidents_by_year_groups.to_frame(name='# of accidents').reset_index()

In [ ]:
accidents_by_year_df.sort_values(['# of accidents'], ascending=False).head(10)

In [ ]:
accident_trend_plot_all = sns.regplot(accidents_by_year_df['Year'],accidents_by_year_df['# of accidents'])
accident_trend_plot_all_figure = accident_trend_plot_all.get_figure()
plt.savefig('output/accident_trend_plot_all.png')

In [ ]:
accidents_by_year.plot()

#### Bicycle Accidents over time

In [ ]:
bicycle_accidents_by_day = bicycle_data.groupby('Date').size()

In [ ]:
bicycle_accidents_by_month = bicycle_accidents_by_day.resample('M').sum()

In [ ]:
bicycle_accidents_by_year = bicycle_accidents_by_day.resample('Y').sum()

In [ ]:
bicycle_accidents_by_day.plot()

In [ ]:
bicycle_accidents_by_month.plot()

In [ ]:
bicycle_accidents_by_year.plot()

In [ ]:
bicycle_accidents_by_year_groups = bicycle_data.groupby('Year').size()
bicycle_accidents_by_year_df = bicycle_accidents_by_year_groups.to_frame(name='# of accidents').reset_index()

In [ ]:
bicycle_accident_trend_plot_all = sns.regplot(bicycle_accidents_by_year_df['Year'],bicycle_accidents_by_year_df['# of accidents'])
bicycle_accident_trend_plot_all_figure = bicycle_accident_trend_plot_all.get_figure()
plt.savefig('output/accident_trend_plot_bicycle.png')

#### Pedestrian Accidents over time

In [ ]:
pedestrian_accidents_by_day = pedestrian_data.groupby('Date').size()

In [ ]:
pedestrian_accidents_by_month = pedestrian_accidents_by_day.resample('M').sum()

In [ ]:
pedestrian_accidents_by_year = pedestrian_accidents_by_day.resample('Y').sum()

In [ ]:
pedestrian_accidents_by_day.plot()

In [ ]:
pedestrian_accidents_by_month.plot()

In [ ]:
pedestrian_accidents_by_year.plot()

In [ ]:
pedestrian_accidents_by_year_groups = pedestrian_data.groupby('Year').size()
pedestrian_accidents_by_year_df = pedestrian_accidents_by_year_groups.to_frame(name='# of accidents').reset_index()

In [ ]:
pedestrian_accident_trend_plot_all = sns.regplot(pedestrian_accidents_by_year_df['Year'],pedestrian_accidents_by_year_df['# of accidents'])
pedestrian_accident_trend_plot_all_figure = pedestrian_accident_trend_plot_all.get_figure()
plt.savefig('output/accident_trend_plot_pedestrian.png')

## Map Data

In [ ]:
# all accidents
df_to_geojson(data, filename='output/all_accidents.geojson',
              properties=['Object 1', 'Object 2', 'Day of Week', 'Year', 'Bicycle Involved', 'Pedestrian Involved', 'No Bike or Pedestrian Involved'],
              lat='Latitude', lon='Longitude', precision=7)

## Some things noticed:
* Big drop in all accidents in 2014, less accidents? or worse reporting?
* Big spike in all accidents in 2015, reporting change? or actual increase? or making up for 2014?
    * weather related, big spike was February of 2015 when the area had major snow storms
* Large number of reported accident coordinates are City Hall, likely what's used when it's not reported or not accurately reported

## Considerations:
* Reporting issues:
    * data is a bit messy, could be a general reflection on in-accuracy of reporting
    * it's possible there is bias in what accidents are reported and when

## Future Work
* Figure out the denominator, i.e. better comparison of locations by looking at accident frequency AND amount of traffic in across locations
* Is it predictable? How likely can you predict accidents based on historical data?